# Writer's Notes

- Adding more datasource (twitter, mediastack)
- Predicting based on classification algorithm (not using the rule based), actually we don't know yet which one is better no?
- Improving the ESG News Filter (does what we have right now is not good enough?)
- Not relying on FinBert? Developing our own new ESG Sentiment?
- give weight on different industries? by integrating MarketPsyched on different industries

---

FinBERT (Financial Bidirectional Encoder Representations from Transformer) is a pretrained transformer model trained on financial corpora. Some of the objective of the Fin-BERT: Sentiment analysis, financial question-answering, and financial document classification.

The typical large dataset of financial documents are news article, datasets of financial documents, regulatory filings, and earning reports.

The model is developed by NVIDIA that is specifically designed to analyse financial text

Reference: [link](https://medium.com/codex/stocks-news-sentiment-analysis-with-deep-learning-transformers-and-machine-learning-cdcdb827fc06#:~:text=Sentiment%20Analysis%20and%20Transformers,model%20trained%20on%20financial%20corpora.)

# 1. Import Libraries

In [1]:
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import pandas as pd
import string
import torch
import warnings

warnings.filterwarnings('ignore')

In [2]:
# dummy dataframe of headline for sentiment result
df_barc = pd.read_csv('../data/Barclays.csv')                 

In [3]:
df_barc.shape

(1351, 4)

# 2. Filter ESG News

In [4]:
# dummy dataframe of headline for sentiment result
esg_glossaries = pd.read_csv('../data/ESG Score Compile - Glossary.csv')                 

# remove unused row number
esg_glossaries = esg_glossaries.drop(['No'], axis=1)

# lower the glossaries
esg_glossaries['Keywords'] = esg_glossaries['Keywords'].str.lower()

# remove duplicates
esg_glossaries = esg_glossaries.drop_duplicates(['Keywords'])

In [5]:
def filter_esg_news(df, esg_glossaries, column, verbose = False):

    ESG_news_filter = pd.DataFrame()
    cnt = 0
    for key in esg_glossaries.Keywords:
        cnt+=1
        ESG_news_filter = pd.concat([ESG_news_filter, df[df[column].str.contains(key)]])

        if verbose:
            if cnt % 10 == 0:
                print(cnt)
            
    return ESG_news_filter

# 3. Barclays

## 3.1 Pre-Processing

### 3.1.1 Headline

In [6]:
def text_preprocessing(df, esg_glossaries, firm='barclays'):
    df_copy = df.copy()

    # assign a new seperate column text_pp for preprocessing
    df_copy['text_pp'] = df_copy.text

    # make the string lower
    df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: x.lower())

    # remove the punctuation
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: x.translate(translator))

    # remove unnecessary word
    df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: x.replace('rpt',''))

    # delete unnecessary double-space
    df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: ' '.join(x.split()))

    # filterout news that contains barclays in the headline news.
    df_copy = df_copy[df_copy.text_pp.str.contains(firm)]
    
    # remove duplicates of headers
    df_copy = df_copy.drop_duplicates(['text_pp'])
    print('preprocessing and drop duplicates news:', df_copy.shape[0])
    
    # filter out only ESG news
    df_copy = filter_esg_news(df_copy, esg_glossaries, 'text_pp')
    print('filter out non-esg news:', df_copy.shape[0])

    return df_copy

In [7]:
df_barc_pp = text_preprocessing(df_barc, esg_glossaries, 'barclays')

preprocessing and drop duplicates news: 670
filter out non-esg news: 45


In [8]:
df_barc_pp.head()

,versionCreated,text,storyId,sourceCode,text_pp
972,2023-05-03 09:29:15+00:00,UPDATE 1-Barclays toughens deforestation rules...,urn:newsml:newswire.refinitiv.com:20230503:nL1...,NS:RTRS,update 1 barclays toughens deforestation rules...
973,2023-05-03 07:36:47+00:00,Barclays toughens deforestation rules for beef...,urn:newsml:newswire.refinitiv.com:20230503:nL8...,NS:RTRS,barclays toughens deforestation rules for beef...
974,2023-05-03 07:36:33+00:00,BARCLAYS TELLS BEEF COMPANIES THEY MUST HAVE A...,urn:newsml:newswire.refinitiv.com:20230503:nL8...,NS:RTRS,barclays tells beef companies they must have a...
868,2022-03-15 18:06:46+00:00,Barclays hires from Moelis for activism defens...,urn:newsml:newswire.refinitiv.com:20220315:nL2...,NS:RTRS,barclays hires from moelis for activism defens...
856,2022-03-22 09:46:59+00:00,BARCLAYS SETS FINAL EXIT DATES ON 'PROGRESSIVE...,urn:newsml:newswire.refinitiv.com:20220322:nL5...,NS:RTRS,barclays sets final exit dates on progressive ...


In [9]:
df_barc_pp = df_barc_pp.reset_index(drop=True)

Preprocessing

- Fix the timestamp
- Look up at the text

### 3.1.2 Fix the timestamp

In [10]:
def timestamp_preprocessing(df):
    
    df_copy = df.copy()
    
    df_copy.versionCreated = df_copy.versionCreated.apply(lambda x: x[:19])
    df_copy.versionCreated = pd.to_datetime(df_copy.versionCreated)
    
    df_copy['year_month'] = df_copy['versionCreated'].apply(lambda x: datetime.strftime(x, '%Y-%m'))
    df_copy['year'] = df_copy['versionCreated'].apply(lambda x: datetime.strftime(x, '%Y'))

    return df_copy

In [11]:
df_barc_pp = timestamp_preprocessing(df_barc_pp)
df_barc_pp = df_barc_pp.sort_values('versionCreated')

## 3.2 Produce sentiment

In [12]:
# create a tokenizer object
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# fetch the pretrained model 
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [13]:
def sentim_analyzer(df, tokenizer, model, headline_colum = 'headline'):
    ''' Given a df that contains a column 'headline' with article healine texts, it runs inference on the healine with the 'model' (FinBert) 
       and inserts output sentiment features into the dataframe in the respective columns (Positive_sentim, Negative_sentim, Neutral_sentim)
       
        Parameters :
          df : A dataframe that contains headlines in a column called 'headline' . 
          tokenizer(AutoTokenizer object) : A pre-processing tokenizer object from Hugging Face lib. 
          model (AutoModelForSequenceClassification object) : A hugging face transformer model.     
          
          returns df : The initial dataframe with the 3 sentiment features as columns for each headline'''
    
    for i in df.index:
        try:
            headline = df.loc[i, headline_colum]
        except:
            return print(f' \'{headline_colum}\' column might be missing from dataframe')
        # Pre-process input phrase
        
        input = tokenizer(headline, padding = True, truncation = True, return_tensors='pt')
        
        # Estimate output
        output = model(**input)
        # Pass model output logits through a softmax layer.
        predictions = torch.nn.functional.softmax(output.logits, dim=-1)
        df.loc[i, 'Positive'] = predictions[0][0].tolist()
        df.loc[i, 'Negative'] = predictions[0][1].tolist()
        df.loc[i, 'Neutral']  = predictions[0][2].tolist()
    # rearrange column order
    try:
        df = df[['date', 'stock', 'Open', 'Close', 'Volume',  'headline', 'Positive', 'Negative', 'Neutral','Price_change']]
    except:
        pass
    return df

### 3.2.1 Run the Sentiment

In [14]:
df_barc_pp = sentim_analyzer(df_barc_pp, tokenizer, model, headline_colum= 'text_pp')

In [15]:
df_barc_pp.head()

,versionCreated,text,storyId,sourceCode,text_pp,year_month,year,Positive,Negative,Neutral
21,2022-02-28 10:02:47,UK'S FCA: FINANCIAL PENALTY ACCOUNTS THAT BARC...,urn:newsml:newswire.refinitiv.com:20220228:nFW...,NS:RTRS,uk s fca financial penalty accounts that barcl...,2022-02,2022,0.667625,0.218301,0.114074
3,2022-03-15 18:06:46,Barclays hires from Moelis for activism defens...,urn:newsml:newswire.refinitiv.com:20220315:nL2...,NS:RTRS,barclays hires from moelis for activism defens...,2022-03,2022,0.105249,0.016328,0.878423
4,2022-03-22 09:46:59,BARCLAYS SETS FINAL EXIT DATES ON 'PROGRESSIVE...,urn:newsml:newswire.refinitiv.com:20220322:nL5...,NS:RTRS,barclays sets final exit dates on progressive ...,2022-03,2022,0.023519,0.034666,0.941815
12,2022-03-28 13:15:01,BUZZ-Barclays shares take a knock from $590 ml...,urn:newsml:newswire.refinitiv.com:20220328:nL3...,NS:RTRS,buzz barclays shares take a knock from 590 mln...,2022-03,2022,0.256782,0.699804,0.043414
44,2022-03-31 12:00:00,EXCLUSIVE-ASCENT RESOURCES LLC WORKING WITH CI...,urn:newsml:newswire.refinitiv.com:20220331:nL2...,NS:RTRS,exclusive ascent resources llc working with ci...,2022-03,2022,0.806450,0.011091,0.182459


In [16]:
df_barc_pp.shape

(45, 10)

In [17]:
columns = ['year_month','year', 'versionCreated','storyId','text_pp', 'Positive', 'Negative', 'Neutral']

In [18]:
df_barc_pp = df_barc_pp[columns]
# df_barc_pp['Positive_Neutral'] = df_barc_pp['Positive'] + df_barc_pp['Neutral']

## 3.3 Aggregate

In [19]:
def grade_calculation(negative):
    pos_ntrl = 1 - negative 
    val = 'A+'

    if pos_ntrl <= 0.083333:
        val = 'D-'
    elif 0.083333 < pos_ntrl <= 0.166666: 
        val = 'D'
    elif 0.166666 < pos_ntrl <= 0.250000: 
        val = 'D+'
    elif 0.250000 < pos_ntrl <= 0.333333: 
        val = 'C-'
    elif 0.333333 < pos_ntrl <= 0.416666: 
        val = 'C'
    elif 0.416666 < pos_ntrl <= 0.500000: 
        val = 'C+'
    elif 0.500000 < pos_ntrl <= 0.583333: 
        val = 'B-'
    elif 0.583333 < pos_ntrl <= 0.666666: 
        val = 'B'
    elif 0.666666 < pos_ntrl <= 0.750000: 
        val = 'B+'
    elif 0.750000 < pos_ntrl <= 0.833333: 
        val = 'A-'
    elif 0.833333 < pos_ntrl <= 0.916666: 
        val = 'A'

    return val

### 3.3.1 Yearly

In [20]:
agg_score_barclays_yearly = df_barc_pp[['year', 'Negative']].groupby(['year']).mean()
agg_score_barclays_yearly['ESG_Controversy'] = agg_score_barclays_yearly.Negative.apply(lambda x: grade_calculation(x))

In [21]:
agg_score_barclays_yearly.iloc[1:]

,Negative,ESG_Controversy
year,,
2023,0.308205,B+


### 3.3.2 Monthly

In [22]:
agg_score_barclays_monthly = df_barc_pp[['year_month', 'Negative']].groupby(['year_month']).mean()

In [23]:
agg_score_barclays_monthly['ESG_Controversy'] = agg_score_barclays_monthly.Negative.apply(lambda x: grade_calculation(x))

In [24]:
agg_score_barclays_monthly

,Negative,ESG_Controversy
year_month,,
2022-02,0.218301,A-
2022-03,0.177214,A-
2022-04,0.008771,A+
2022-06,0.037077,A+
2022-07,0.367655,B
2022-09,0.322622,B+
2022-10,0.671028,C-
2022-11,0.794297,D+
2023-01,0.350465,B


## 3.4 Proof

In [25]:
proof_df_1 = df_barc_pp[df_barc_pp['year_month'] == '2022-03']

In [26]:
proof_df_1 = proof_df_1[['year_month', 'text_pp', 'Positive', 'Negative', 'Neutral']]

In [27]:
for i in range(proof_df_1.shape[0]):
    print(proof_df_1.iloc[i, 1])

barclays hires from moelis for activism defense esg group
barclays sets final exit dates on progressive phase out of thermal coal financing
buzz barclays shares take a knock from 590 mln loss from mishandled product sales
exclusive ascent resources llc working with citigroup inc c n and barclays plc barc l on potential ipo that may value u s gas producer at 6 bln sources
barclays raises u s minimum hourly wage to 20 50


## 3.5 Caveat

---

In [28]:
df_barc_pp[df_barc_pp['Positive'] > 0.7].head()

,year_month,year,versionCreated,storyId,text_pp,Positive,Negative,Neutral
44,2022-03,2022,2022-03-31 12:00:00,urn:newsml:newswire.refinitiv.com:20220331:nL2...,exclusive ascent resources llc working with ci...,0.806450,0.011091,0.182459
15,2022-04,2022,2022-04-28 06:01:47,urn:newsml:newswire.refinitiv.com:20220428:nTU...,barclays barc l legacy loan portfolio a custom...,0.796807,0.009103,0.194089
20,2022-04,2022,2022-04-28 06:01:47,urn:newsml:newswire.refinitiv.com:20220428:nTU...,barclays barc l legacy loan portfolio a custom...,0.796807,0.009103,0.194089
10,2022-06,2022,2022-06-24 06:00:36,urn:newsml:newswire.refinitiv.com:20220624:nTU...,barclays acquisition will enable barclays to b...,0.898236,0.006863,0.094901
40,2022-06,2022,2022-06-30 12:15:08,urn:newsml:newswire.refinitiv.com:20220630:nL8...,corrected unite increase targeted towards lowe...,0.871343,0.050939,0.077718


In [29]:
df_barc_pp.tail()

,year_month,year,versionCreated,storyId,text_pp,Positive,Negative,Neutral
2,2023-05,2023,2023-05-03 07:36:33,urn:newsml:newswire.refinitiv.com:20230503:nL8...,barclays tells beef companies they must have a...,0.295584,0.016970,0.687446
1,2023-05,2023,2023-05-03 07:36:47,urn:newsml:newswire.refinitiv.com:20230503:nL8...,barclays toughens deforestation rules for beef...,0.187678,0.101993,0.710329
0,2023-05,2023,2023-05-03 09:29:15,urn:newsml:newswire.refinitiv.com:20230503:nL1...,update 1 barclays toughens deforestation rules...,0.262882,0.182026,0.555093
6,2023-05,2023,2023-05-03 11:47:43,urn:newsml:newswire.refinitiv.com:20230503:nFW...,barclays tightened thermal coal policy now in ...,0.572601,0.054436,0.372963
5,2023-05,2023,2023-05-03 12:34:25,urn:newsml:newswire.refinitiv.com:20230503:nFW...,brief barclays says it has tightened its therm...,0.050023,0.821592,0.128385


In [ ]:
df_barc_pp.loc[931]['text_pp'][0]

In [ ]:
df_barc_pp.loc[931]['Positive']

In [ ]:
df_barc_pp.loc[931]['Negative']

In [ ]:
df_barc_pp.loc[931]['Neutral']

---